In [1]:
# Import dependencies
import pandas as pd
from datetime import datetime
import time
from splinter import Browser
from bs4 import BeautifulSoup

In [ ]:
# Set path and browser for scraping Yahoo website
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://finance.yahoo.com/quote/VBMFX/history?period1=' + period1 + '&period2=' + period2 + '&interval=1mo&filter=history&frequency=1mot'
start_date = "01/01/1994"
period1 = datetime.datetime.strptime(start_date, "%d/%m/%Y").timestamp()
period2 = datetime.timestamp(datetime.now)
# browser.visit(url)

# html = browser.html
# soup= BeautifulSoup(html, 'html.parser')

print(period1)
print(period2)

In [ ]:
# # Using Alpha Vantange API - Find data for SPY ticker

# import requests
# import json
# from config import apikey

# data_frequency = "MONTHLY"
# ticker = "SPY"

# url = f'https://www.alphavantage.co/query?function=TIME_SERIES_{data_frequency}&symbol={ticker}&apikey=' + apikey
# response = requests.get(url).json()

# SPY_dates = []
# SPY_close = []
# SPY_volume = []

# symbol = response["Meta Data"]["2. Symbol"]

# for data in response:
#     for date in dates:
#         SPY_dates.append(response["Monthly Time Series"][dates])
#         SPY_close.append(response["Monthly Time Series"][dates]["4. close"])
#         SPY_volume.append(response["Monthly Time Series"][dates]["5. volume"])

# print(symbol)
# print(SPY_dates)

In [2]:
# Using Alpha Vantange API - Find data for SPY ticker

import requests
import json
from config import apikey

data_frequency = "MONTHLY"
ticker = "SPY"

url = f'https://www.alphavantage.co/query?function=TIME_SERIES_{data_frequency}&symbol={ticker}&apikey=' + apikey
response = requests.get(url).json()

In [3]:
symbol = response["Meta Data"]["2. Symbol"]

print(symbol)

SPY


In [22]:
SPY_dates = []
SPY_close = []
SPY_volume = []

for dates in response["Monthly Time Series"]:
    
        SPY_close.append(response["Monthly Time Series"][dates]["4. close"])
        SPY_volume.append(response["Monthly Time Series"][dates]["5. volume"])
        
#         for key, value in response["Monthly Time Series"][dates]:
#             SPY_dates.append(value)
            
print(dates)

2000-01-31


In [9]:
print(SPY_close)

['321.8600', '314.3100', '303.3300', '296.7700', '292.4527', '297.4300', '293.0000', '275.2700', '294.0200', '282.4800', '278.6800', '269.9300', '249.9200', '275.6500', '270.6300', '290.7200', '290.3100', '281.3300', '271.2800', '270.9400', '264.5100', '263.1500', '271.6500', '281.9000', '266.8600', '265.0100', '257.1500', '251.2300', '247.4900', '246.7700', '241.8000', '241.4400', '238.0800', '235.7400', '236.4700', '227.5300', '223.5300', '220.3800', '212.5500', '216.3000', '217.3800', '217.1200', '209.4750', '209.8400', '206.3308', '205.5200', '193.5600', '193.7208', '203.8700', '208.6900', '207.9300', '191.5900', '197.5400', '210.4500', '205.8900', '211.1400', '208.4600', '206.4300', '210.6600', '199.4500', '205.5400', '207.2000', '201.6600', '197.0200', '200.7100', '193.0900', '195.7200', '192.6800', '188.3100', '187.0100', '186.2900', '178.1800', '184.6900', '181.0000', '175.7900', '168.0100', '163.6500', '168.7100', '160.4200', '163.4500', '159.6800', '156.6700', '151.6100', '14

In [10]:
print(SPY_volume)

['1268511696', '1035047008', '1404743524', '1322797499', '2038348125', '1135591762', '1340435548', '1845593139', '1209204906', '1678081316', '1371716235', '2048691645', '3102621849', '2021061342', '3024346032', '1228083461', '1308443632', '1266961027', '1600700314', '1606397329', '2000427231', '2336204952', '2923721958', '1996090155', '1715222830', '1364516581', '1320624558', '1286405280', '1557031716', '1062993341', '1572752876', '1494781355', '1549587787', '1921473790', '1365136521', '1482392744', '1821800274', '2073824327', '1725687191', '2303701186', '1519703229', '1648453879', '2630619702', '1834562512', '1910635495', '2324144031', '2920730619', '3712951060', '2924836766', '1924512621', '2536184532', '3279512868', '3734845183', '2452757863', '2535242753', '1892519799', '2036685271', '2749923019', '1901638015', '3183506003', '3045170843', '1571225009', '3943067849', '2106580212', '1829764370', '1893838621', '1675360233', '1780157245', '2345807033', '2553782755', '2394183674', '2530